# Workflow to Validate NISAR L2 Coseismic Displacement Requirement

**Original code authored by:** David Bekaert, Heresh Fattahi, Eric Fielding, and Zhang Yunjun 

Extensive modifications by Adrian Borsa and Amy Whetter 2022

Subsequent updates and reformatting by Rob Zinke, Katia Tymofyeyeva and Adrian Borsa 2025

<div class="alert alert-warning">
Step 1 of the validation workflow is executed in the **ARIA_prep** notebook. All processing steps in this notebook MUST be run sequentially, although several are optional and can be skipped.
</div>

<hr/>

<hr/>

## Table of Contents:
<a id='example_TOC'></a>

[**Environment Setup**](#setup)

[**1. Download and Prepare Interferograms**](#prep_ifg)

[**2. Generation of Time Series from Interferograms**](#gen_ts)
- [2.1. Validate/Modify Interferogram Network](#validate_network)
- [2.2. Generate Quality Control Mask](#generate_mask)
- [2.3. Reference Interferograms To Common Lat/Lon](#common_latlon)
- [2.4. Invert for SBAS Line-of-Sight Timeseries](#invert_SBAS)

[**3. Optional Corrections**](#opt_correction)
- [3.1. Solid Earth Tides Correction](#solid_earth)
- [3.2. Tropospheric Delay Correction](#tropo_corr)
- [3.3. Phase Deramping ](#phase_deramp)
- [3.4. Topographic Residual Correction ](#topo_corr) 

[**4. Decomposition of InSAR and GNSS Time Series into Basis Functions**](#decomp_ts)
- [4.1. Estimate InSAR LOS Velocities](#insar_vel)
- [4.2. Estimate InSAR Coseismic Displacement](#co_step)
- [4.3. Find Collocated GNSS Stations](#find_gps)  
- [4.4. Get GNSS Position Time Series](#gps_ts) 
- [4.5. Make GNSS LOS Velocities/Displacements](#gps_los)
- [4.6. Re-reference GNSS and InSAR LOS Coseismic Step](#reference)

[**5. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison**](#validation1)
- [5.1. Make GNSS-InSAR Displacement Residuals at GNSS Station Locations](#make_resids)
- [5.2. Make Double-differenced Displacement Residuals](#make_ddiff)
- [5.3. Amplitude vs. Distance of Double-differences (not quite a structure function)](#amp_vs_dist)

[**6. NISAR Validation Approach 2: InSAR-only Structure Function**](#nisar_validation2)
- [6.1. Read Array and Mask Pixels with no Data](#array_mask)
- [6.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend](#remove_trend)
- [6.3. Coseismic Requirement Validation: Method 2](#validation2)

[**Appendix: Supplementary Comparisons and Plots**](#appendix)
- [A.1. Compare Raw Velocities](#compare_raw)
- [A.2. Plot Velocity Residuals](#plot_residuals)
- [A.3. Plot Double-differenced Residuals](#plot_ddiff)
- [A.4. GNSS Time-series Plots](#plot_timeseries)

<hr/>

## Environment Setup<a id='#setup'></a>

### Load Python Packages<a id='#load_packages'></a>

In [ ]:
import os
import json
import subprocess
import warnings
from datetime import datetime as dt, timedelta

import numpy as np
from scipy import signal
from matplotlib import pyplot as plt
import matplotlib.patches as patches

from mintpy.cli import plot_network, view
from mintpy.utils import readfile, ptime, time_func, utils as ut
from mintpy.objects import gnss, timeseries
from mintpy.smallbaselineApp import TimeSeriesAnalysis

from solid_utils.gnss_utils import scale_gnss_m_to_mm
from solid_utils.fitting import IterativeOutlierFit
from solid_utils.sampling import SiteDisplacement, load_geo, samp_pair, profile_samples, haversine_distance
from solid_utils.plotting import display_coseismic_validation, display_validation_table
from solid_utils.saving import save_results
from solid_utils.configs import update_reference_point
from solid_utils.corrections import run_cmd, apply_cor

#Set Global Plot Parameters
plt.rcParams.update({'font.size': 12})

### Define Calval Site and Parameters<a id='set_calval_params'></a>

In [ ]:
# === Basic Configuration ===
site='test' # Cal/Val location ID from my_sites.txt
requirement = 'Coseismic' # Options: 'Secular' 'Coseismic' 'Transient'
dataset = "ARIA_S1_new"  # Dataset type: 'ARIA_S1', 'ARIA_S1_new'
aria_gunw_version = "3_0_1"

rundate = "20250826"  # Date of this Cal/Val run
version = "1"         # Version of this Cal/Val run
custom_sites = "/home/jovyan/my_sites.txt"  # Path to custom site metadata

# === Username Detection / Creation ===
user_file = "/home/jovyan/me.txt"
if os.path.exists(user_file):
    with open(user_file, "r") as f:
        you = f.readline().strip()
else:
    you = input("Please type a username for your Cal/Val outputs: ").strip()
    with open(user_file, "w") as f:
        f.write(you)

# === Load Cal/Val Site Metadata ===
try:
    with open(custom_sites, "r") as f:
        sitedata = json.load(f)
    site_info = sitedata["sites"][site]
except (FileNotFoundError, json.JSONDecodeError) as e:
    raise RuntimeError(f"Failed to load site metadata from {custom_sites}: {e}")
except KeyError:
    raise ValueError(f"Site ID '{site}' not found in {custom_sites}")

print(f"Loaded site: {site}")

# === Plot Parameters ===
vmin, vmax = -25, 25  # mm/yr
cmap_str = 'RdBu_r'
cmap = plt.get_cmap(cmap_str)

### Set Directories and Files<a id='set_directories'></a>

In [ ]:
# === Define Cal/Val Directory Structure ===
BASE_DIR = "/scratch/nisar-st-calval-solidearth"
site_dir = os.path.join(BASE_DIR, dataset, site)
work_dir = os.path.join(site_dir, requirement, you, rundate, f"v{version}")
gunw_dir = os.path.join(site_dir, "products")
mintpy_dir = os.path.join(work_dir, "MintPy")
weather_dir = os.path.join(site_dir)

# === Home directory for saving reports ===
home_dir = os.path.join("/home/jovyan/validation/", site, requirement, rundate, f"v{version}")
if not os.path.exists(home_dir):
    os.makedirs(home_dir)

# === Log Directory Paths ===
print(f"  Work directory: {work_dir}")
print(f"  GUNW directory: {gunw_dir}")
print(f"MintPy directory: {mintpy_dir}")

# === Check MintPy Directory Existence ===
if not os.path.exists(mintpy_dir):
    print("\nERROR: Stop! MintPy processing directory is missing.")
    print("This may indicate the prep notebook has not been run.")
    print("Missing path:", mintpy_dir, "\n")
else:
    os.chdir(mintpy_dir)

# === Set Expected MintPy Filenames ===
geom_file = os.path.join(mintpy_dir, "inputs", "geometryGeo.h5")
msk_file  = os.path.join(mintpy_dir, "maskTempCoh.h5")  # alt: maskConnComp.h5
ts_file   = os.path.join(mintpy_dir, "timeseries.h5")
vel_file  = os.path.join(mintpy_dir, "velocity.h5")
config_file = os.path.join(mintpy_dir, site_info.get('calval_location') + '.cfg')

In [ ]:
configs = readfile.read_template(config_file)
print('#' * 10, "MintPy Configs", '#' * 10)
for key, value in configs.items():
    print(f"{key}: {value}")

<br>
<hr>

<a id='#prep_ifg'></a>
## 1. Download and Prepare Interferograms

Executed in *ARIA_prep* notebook

<br>
<hr>

<a id='#gen_ts'></a>
## 2. Generation of Time Series from Interferograms

<a id='validate_network'></a>
### 2.1. Validate/Modify Interferogram Network

Add additional parameters to config_file in order to remove selected interferograms, change minimum coherence, etc.

In [ ]:
# Formulate ifgramStack file
ifgramStack_file = os.path.join(mintpy_dir, 'inputs/ifgramStack.h5')

# Modify network - base command
command = f"modify_network.py {ifgramStack_file} -t {config_file}"

# Check whether exclusions specified in my_sites file
if site_info.get('ifgExcludePair') not in [None, 'auto', 'no']:
    command += f" --exclude-ifg {site_info.get('ifgExcludePair')}"

if site_info.get('ifgExcludeDate') not in [None, 'auto', 'no']:
    command += f" --exclude-date {site_info.get('ifgExcludeDate')}"

if site_info.get('ifgExcludeIndex') not in [None, 'auto', 'no']:
    command += f" --exclude-ifg-index {site_info.get('ifgExcludeIndex')} "

# Run command
process = subprocess.run(command, shell=True)
plot_network.main([ifgramStack_file])

<a id='generate_mask'></a>
### 2.2. Generate Quality Control Mask


Mask files can be can be used to mask pixels in the time-series processing. Below we generate an initial mask file `maskConnComp.h5` based on the connected components for all the interferograms, which is a metric for unwrapping quality. After time-series analysis is complete, we will calculate a mask from the temporal coherence or variation of phase or displacement with time to make `maskTempCoh.h5`.

In [ ]:
command='generate_mask.py inputs/ifgramStack.h5  --nonzero  -o maskConnComp.h5  --update'
process = subprocess.run(command, shell=True)
view.main(['maskConnComp.h5', 'mask'])

<a id='common_latlon'></a>
### 2.3. Reference Interferograms To Common Lat/Lon


In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True, shell=True)
os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

### 2.3.1 Change Reference Point (Optional) <a id='common_latlon_change'></a>

Use the cells below to update the reference point. Uncomment and enter the desired *new_lat* and *new_lon*, then verify the update in the output.

In [ ]:
if site_info.get('reference_lalo') != 'auto':
    new_lat = site_info.get('reference_lalo').split(',')[0]
    new_lon = site_info.get('reference_lalo').split(',')[1]
    update_reference_point(config_file, new_lat, new_lon) # updates the reference point in MintPy config file
    
# Now reference interferograms to common lat/lon
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep reference_point'
process = subprocess.run(command, shell=True)
os.system('info.py inputs/ifgramStack.h5 | egrep "REF_"');

<a id='invert_SBAS'></a>
### 2.4. Invert for SBAS Line-of-Sight Timeseries


In [ ]:
command = 'smallbaselineApp.py ' + str(config_file) + ' --dostep invert_network'
process = subprocess.run(command, shell=True)
timeseries_filename = f"{mintpy_dir}/timeseries.h5"

In [ ]:
velocity_filename = f"{mintpy_dir}/velocity.h5"
command = 'timeseries2velocity.py ' + timeseries_filename + ' -o ' + velocity_filename
process = subprocess.run(command, shell=True)

In [ ]:
# Visualize velocity before any corrections
title = 'LOS velocity, no corrections'
view_opts = [
    velocity_filename, "velocity",
    "-m", msk_file,
    "--dem", f"{mintpy_dir}/inputs/geometryGeo.h5", "--dem-blend", "--dem-nocontour",
    "--lalo-label",
    "--coastline", "10m",
    "-c", cmap_str,
    "-u", "mm",
    "-v", str(vmin), str(vmax),
    "--title", title,
    "--noverbose"
]
view.main(view_opts)

<br>
<hr>

<a id='opt_correction'></a>
## 3. Optional Corrections

Phase distortions related to solid earth and ocean tidal effects as well as those due to temporal variations in the vertical stratification of the atmosphere can be mitigated using the approaches described below. At this point, it is expected that these corrections will not be needed to validate the mission requirements, but they may be used to produce the highest quality data products. Typically, these are applied to the estimated time series product rather than to the individual interferograms, since they are a function of the time of each radar acquisition.

<a id='solid_earth'></a>
### 3.1. Solid Earth Tides Correction

Optional solid Earth tide correction is included in the ARIA GUNW products and can be extracted using the <i>ARIA_prep</i> notebook.  
Once extracted, you can calculate the correction using the cells below.

In [ ]:
if 'do_SET' in site_info.keys() and site_info.get('do_SET') != "False":
    if site_info.get('SET_source') == 'aria':
        set_cor_file = f"{mintpy_dir}/inputs/solidEarthTide_ARIA.h5"     # Input SET correction file
    else:
        set_cor_file = f"{mintpy_dir}/inputs/SET.h5"

    dirpath, filename = os.path.split(timeseries_filename)
    name, ext = os.path.splitext(filename)
    output_timeseries = os.path.join(dirpath, f"{name}_SET{ext}")

    dirpath, filename = os.path.split(velocity_filename)
    name, ext = os.path.splitext(filename)
    output_velocity = os.path.join(dirpath, f"{name}_SET{ext}")
else: 
    site_info['do_SET'] = "False"
    print('#'*10, 'Solid Earth Tide Correction set to False', '#'*10)

In [ ]:
# Apply correction if flag is set to True
if site_info.get('do_SET') != "False":
    timeseries_filename, velocity_filename = apply_cor('SET', timeseries_filename, set_cor_file, config_file, mintpy_dir, output_timeseries, output_velocity)
else:
    print('#'*10, 'Solid Earth Tide Correction set to False', '#'*10)

In [ ]:
# Visualize the corrections
if site_info.get('do_SET') != "False":
    view_opts = [
        set_cor_file,
        "-m", msk_file,
        "-c", cmap_str,
        "--noverbose"
    ]
    view.main(view_opts)
else: 
    print('#'*10, 'Solid Earth Tide Correction set to False', '#'*10)

In [ ]:
# Visualize velocity
print(velocity_filename)
if site_info.get('do_SET') != "False":
    view_opts = [
        velocity_filename, "velocity",
        "-m", msk_file,
        "--dem", f"{mintpy_dir}/inputs/geometryGeo.h5",
        "--dem-blend",         # blend the DEM shade with input image to have a GMT-like impression.
        "--dem-nocontour",     # do not show DEM contour lines
        "--coastline", "10m", # Draw coastline with specified resolution (10m, 50m, 110m). This enables '--lalo-label'
        "--lalo-label",        # Show N, S, E, W tick label for plot in geo-coordinate.
        "-c", cmap_str,
        "-u", "mm",
        "-v", str(vmin), str(vmax),
        "--title", 'Velocity after correction for SET',
        "--noverbose"
    ]
    view.main(view_opts)
else: 
    print('#'*10, 'Solid Earth Tide Correction set to False', '#'*10)

### 3.2 Ionosphere Correction

In [ ]:
if 'do_iono' in site_info.keys() and site_info.get('do_iono') != "False":
    iono_stack_file = f"{mintpy_dir}/inputs/ionStack.h5"             # Input ionosphere stack file  

    dirpath, filename = os.path.split(timeseries_filename)
    name, ext = os.path.splitext(filename)
    output_timeseries = os.path.join(dirpath, f"{name}_iono{ext}")
    output_velocity = os.path.join(dirpath, f"velocity.h5")
else:
    site_info['do_iono'] = "False"
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

In [ ]:
# Apply correction if flag is set to True
if site_info.get('do_iono')!= "False":
    timeseries_filename, velocity_filename = apply_cor('iono', timeseries_filename, iono_stack_file, config_file, mintpy_dir, output_timeseries, output_velocity)
    iono_cor_file = os.path.join(mintpy_dir, "ion.h5")
else:
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

In [ ]:
# Visualize the corrections
if site_info.get('do_iono') != "False":
    view_opts = [
        iono_stack_file,
        "--wrap",
        "-m", msk_file,
        "-c", cmap_str,
        "--noverbose"
    ]
    print('#'*10, 'Wrapped pair-wise ionosphere', '#'*10)
    view.main(view_opts)

    view_opts = [
        iono_stack_file,
        "-m", msk_file,
        "-c", cmap_str,
        "--noverbose"
    ]
    print('#'*10, 'Unwrapped pair-wise ionosphere', '#'*10)
    view.main(view_opts)
else: 
    print('#'*10, 'Ionosphere Correction set to False', '#'*10)

<a id='tropo_corr'></a>
### 3.3. Tropospheric Delay Correction

Optional atmospheric correction utilizes the PyAPS (Jolivet et al., 2011, Jolivet and Agram, 2012) module within GIAnT (or eventually a merged replacement for GIAnT and MintPy). PyAPS is well documented, maintained and can be freely downloaded. PyAPS is included in GIAnT distribution). PyAPS currently includes support for ECMWF’s ERA-Interim, NOAA’s NARR and NASA’s MERRA weather models. A final selection of atmospheric models to be used for operational NISAR processing will be done during Phase C.

[T]ropospheric delay maps are produced from atmospheric data provided by Global Atmospheric Models. This method aims to correct differential atmospheric delay correlated with the topography in interferometric phase measurements. Global Atmospheric Models (hereafter GAMs)... provide estimates of the air temperature, the atmospheric pressure and the humidity as a function of elevation on a coarse resolution latitude/longitude grid. In PyAPS, we use this 3D distribution of atmospheric variables to determine the atmospheric phase delay on each pixel of each interferogram.

The absolute atmospheric delay is computed at each SAR acquisition date. For a pixel a_i at an elevation z at acquisition date i, the four surrounding grid points are selected and the delays for their respective elevations are computed. The resulting delay at the pixel a_i is then the bilinear interpolation between the delays at the four grid points. Finally, we combine the absolute delay maps of the InSAR partner images to produce the differential delay maps used to correct the interferograms.

REFERENCE: [https://github.com/insarlab/pyaps#2-account-setup-for-era5]
Read Section 2 for ERA5 (link above) to create an account for the CDS website.

In [ ]:
if 'do_tropo' in site_info.keys() and site_info.get("do_tropo") != "False":
    
    if site_info.get("tropo_model") != "HRRR":
        # Run ERA5-based correction
        dirpath, filename = os.path.split(timeseries_filename)
        name, ext = os.path.splitext(filename)
        output_timeseries = os.path.join(dirpath, f"{name}_ERA5{ext}")

        dirpath, filename = os.path.split(velocity_filename)
        name, ext = os.path.splitext(filename)
        output_velocity = os.path.join(dirpath, f"{name}_ERA5{ext}")

        tropo_cmd = f"tropo_pyaps3.py -f {timeseries_filename} -g {mintpy_dir}/inputs/geometryGeo.h5 -w {weather_dir} -o {output_timeseries}"
        run_cmd(tropo_cmd, desc="Applying ERA5 tropospheric correction")

        tropo_vel_cmd = f"timeseries2velocity.py {output_timeseries} -o {output_velocity}"
        run_cmd(tropo_vel_cmd, desc="Estimating velocity from ERA5-corrected time series")

        tropo_cor_file = os.path.join(mintpy_dir, "inputs", "ERA5.h5")
        timeseries_filename = output_timeseries
        velocity_filename = output_velocity

    else:
        # HRRR-based correction
        tropo_cor_file = os.path.join(mintpy_dir, "inputs", "HRRR_ARIA.h5")

        dirpath, filename = os.path.split(timeseries_filename)
        name, ext = os.path.splitext(filename)
        output_timeseries = os.path.join(dirpath, f"{name}_HRRR{ext}")

        timeseries_filename, velocity_filename = apply_cor(
            cor_name="tropo",
            ts_file=timeseries_filename,
            cor_file=tropo_cor_file,
            config_file=config_file,
            mintpy_dir=mintpy_dir,
            output_ts=output_timeseries,
            output_vel=output_velocity
        )
else:
    site_info['do_tropo'] = "False"
    print("#" * 10, "Troposphere Correction set to False", "#" * 10)


In [ ]:
# Visualize the corrections
if site_info.get('do_tropo') != "False":
    view_opts = [
        tropo_cor_file,
        "-m", msk_file,
        "-c", cmap_str,
        "--noverbose"
    ]
    view.main(view_opts)
else: 
    print('#'*10, 'Troposhere Correction set to False', '#'*10)

<a id='phase_deramp'></a>
### 3.3. Phase Deramping


In [ ]:
try:
    deramp_method = configs['mintpy.deramp']
    ramp_cmd = f"remove_ramp.py {timeseries_filename} -m {msk_file} -s {deramp_method} -o {timeseries_filename}_ramp.h5"
    subprocess.run(ramp_cmd, shell=True)
    timeseries_filename = f"{timeseries_filename}_ramp.h5"
except KeyError:
    print("#" * 10, "No deramping", "#" * 10)

<a id='topo_corr'></a>
### 3.4. Topographic Residual Correction

In [ ]:
try:
    do_demErr = configs['mintpy.topographicResidual']
    if configs['mintpy.topographicResidual.polyOrder'] == 'auto':
        demErr_polyorder = 2
    else:
        demErr_polyorder = configs['mintpy.topographicResidual.polyOrder']
    demErr_cmd = f"dem_error.py {timeseries_filename} -g {mintpy_dir}/inputs/geometryGeo.h5 -p {demErr_polyorder} -o {timeseries_filename}_demErr.h5"
    subprocess.run(demErr_cmd, shell=True)
    timeseries_filename = f"{timeseries_filename}_demErr.h5"
except KeyError:
    print("#" * 10, "No DEM error correction", "#" * 10)

<br>
<hr>

<a id='decomp_ts'></a>
## 4. Decomposition of InSAR and GNSS Time Series into Basis Functions

<a id='insar_vel'></a>
### 4.1. Estimate InSAR LOS Velocities

Given a time series of InSAR LOS displacements, the observations for a given pixel, $U(t)$, can be parameterized as:

$$ U(t) = a \;+\; vt \;+\; c_1 cos (\omega_1t - \phi_{1,}) \;+\; c_2 cos (\omega_2t - \phi_2) \;+\; \sum_{j=1}^{N_{eq}} \left( h_j+f_j F_j (t-t_j) \right)H(t - t_j) \;+\; \frac{B_\perp (t)}{R sin \theta}\delta z \;+\; residual$$ 

which includes a constant offset $(a)$, velocity $(v)$, and amplitudes $(c_j)$ and phases $(\phi_j)$ of annual $(\omega_1)$ and semiannual $(\omega_2)$ sinusoidal terms.  Where needed we can include additional complexity, such as coseismic and postseismic processes parameterized by Heaviside (step) functions $H$ and postseismic functions $F$ (the latter typically exponential and/or logarithmic).   $B_\perp(t)$, $R$, $\theta$, and $\delta z$ are, respectively, the perpendicular component of the interferometric baseline relative to the first date, slant range distance, incidence angle and topography error correction for the given pixel. 

Thus, given either an ensemble of interferograms or the output of SBAS (displacement vs. time), we can write the LSQ problem as 

$$ \textbf{G}\textbf{m} = \textbf{d}$$

where $\textbf{G}$ is the design matrix (constructed out of the different functional terms in Equation 2 evaluated either at the SAR image dates for SBAS output, or between the dates spanned by each pair for interferograms), $\textbf{m}$ is the vector of model parameters (the coefficients in Equation 2) and $\textbf{d}$ is the vector of observations.  For GPS time series, $\textbf{G}, \textbf{d}, \textbf{m}$ are constructed using values evaluated at single epochs corresponding to the GPS solution times, as for SBAS InSAR input. 

With this formulation, we can obtain InSAR velocity estimates and their formal uncertainties (including in areas where the expected answer is zero). 

The default InSAR velocity fit in MintPy is to estimate a mean linear velocity $(v)$ in in the equation, which we do below. This is the same linear velocity used in the Secular notebook.

<div class="alert alert-warning">
If the time-series span less than one year, then annual and semi-annual fuctions might not be reliably fit for noisy data. In that case, the periodic functions will not be included in these analyses.

Note that this workflow **requires** an earthquake date to be specified.
</div>

In [ ]:
# Check that an earthquake date is specified and retrieve that date
if 'earthquakeDate' in sitedata['sites'][site].keys():
    EQdate = [sitedata['sites'][site]['earthquakeDate']]
    print('Earthquake date: {:s}'.format(sitedata['sites'][site]['earthquakeDate']))
else:
    EQdate = []
    warnings.warn('No earthquake date specified. This data set might not be suitable for the coseismic workflow.')

# Specify basis functions
# Changing these might change the meaning of this exercise! Use caution if altering.
ts_functions = {
                'polynomial': 1,
                'periodic': [],  # Periodic terms are in units of years
                'stepDate': EQdate,
                'polyline': [],
                'exp': {},
                'log': {}
}

# Determine time span to check whether fitting of periodic functions is appropriate
ts_metadata = readfile.read_attribute(timeseries_filename)
start_date = dt.strptime(sitedata['sites'][site]['download_start_date'], '%Y%m%d')
end_date = dt.strptime(sitedata['sites'][site]['download_end_date'], '%Y%m%d')
ts_timespan = (end_date - start_date).days

if (ts_timespan < 365*2) and len(ts_functions['periodic']) > 0:
    print('Time span is {:d} days'.format(ts_timespan))
    if ts_timespan < 365:
        ts_functions['periodic'] = []
        print('No periodic functions will be fit')
    else:
        warnings.warn('Warning: Periodic fits might be unreliable for series less than 2 years')

In [ ]:
# Parse the TS model into argument string for command line execution
function_str = ''

# Loop through basis functions
for fcn in ts_functions.keys():
    arg = ts_functions[fcn]
    
    if fcn == 'polynomial':
        function_str += f' --polynomial {arg:d}'
    
    else:
        # Check whether values are passed
        if len(arg) > 0:
            fcn = 'step' if fcn == 'stepDate' else fcn
            # Append basis function name
            function_str += f' --{fcn:s}'

            # Append function arguments (e.g, period, decay, etc.)
            if type(arg) == list:
                function_str += ' {:s}'.format(' '.join([str(a) for a in arg]))
            elif type(arg) == dict:
                for event_date in arg.keys():
                    function_str += f' {event_date:s}'
                    for val in arg[event_date]:
                        function_str += f' {val:f}'

# Run command
vel_file = 'velocity_nostep.h5'
command = 'timeseries2velocity.py ' + timeseries_filename + ' -o ' + vel_file + function_str
process = subprocess.run(command, shell=True)

# Load velocity file
insar_velocities,_ = readfile.read(vel_file, datasetName = 'velocity')  # read velocity file
insar_velocities = insar_velocities * 1000.  # convert InSAR velocities from m/yr to mm/yr

# Set masked pixels to NaN
msk,_ = readfile.read(msk_file)
insar_velocities[msk == 0] = np.nan
insar_velocities[insar_velocities == 0] = np.nan

Now we plot the mean linear velocity fit. The MintPy `view` module automatically reads the temporal coherence mask `maskTempCoh.h5` and applies that to mask out pixels with unreliable velocities (white). Because there is a large coseismic displacement at the time of the earthquake but we are only estimating a single linear velocity, it absorbs the coseismic signal in this plot.

In [ ]:
# Visualize velocity
title = "LOS velocity, linear fit"
view_opts = [
    vel_file, "velocity",
    "-m", msk_file,
    "--dem", f"{mintpy_dir}/inputs/geometryGeo.h5", "--dem-blend", "--dem-nocontour",
    "--lalo-label",
    "--coastline", "10m",
    "-c", cmap_str,
    "-u", "mm",
    "-v", str(vmin), str(vmax),
    "--title", title,
    "--noverbose"
]
view.main(view_opts)

<div class="alert alert-info">
<b>Note :</b> 
Negative values indicates that target is moving away from the radar (i.e., Subsidence in case of vertical deformation).
Positive values indicates that target is moving towards the radar (i.e., uplift in case of vertical deformation). 
</div>

<a id='co_step'></a>
### 4.2. Estimate InSAR Coseismic Displacement

We can use the same time series to estimate the coseismic displacement as a Heaviside $H$ or step function at the time of an earthquake or a number $N_{eq}$ of earthquakes. In the above equation this is the $$\sum_{j=1}^{N_{eq}} \left( h_j+f_j F_j (t-t_j) \right)H(t - t_j) \;$$ set of terms. For simplicity, we consider only one earthquake and we assume the postseismic displacement functions $F$ are small compared to the coseismic displacements, so we only need to solve for the coefficient $h$ of each interferogram pixel.

We call the MintPy `timeseries2velocity.py` program again and specify the time of the earthquake $t_j$. The fit will also include the linear velocity rate separated from the step function amplitude. Both estimated coefficients have their associated uncertainties.

In [ ]:
vel_file = "velocity_coseismic.h5"
command = 'timeseries2velocity.py ' + timeseries_filename + ' -o ' + vel_file + function_str
process = subprocess.run(command, shell=True)

EQdataset = 'step' + sitedata['sites'][site]['earthquakeDate']
EQstep, _ = readfile.read(vel_file, datasetName = EQdataset)

# Set masked pixels to NaN
msk,_ = readfile.read(msk_file)
EQstep[msk == 0] = np.nan
EQstep[EQstep == 0] = np.nan

# Convert coseismic step from m to mm
EQstep *= 1000.

Now we can view the step function amplitude and the new linear velocity estimate.

In [ ]:
# Visualize velocity
title = "LOS Coseismic, earthquake date"
view_opts = [
    vel_file, "step"+sitedata['sites'][site]['earthquakeDate'],
    "-m", msk_file,
    "--dem", f"{mintpy_dir}/inputs/geometryGeo.h5", "--dem-blend", "--dem-nocontour",
    "--lalo-label",
    "--coastline", "10m",
    "-c", cmap_str,
    "-u", "mm",
    "-v", str(vmin), str(vmax),
    "--title", title,
    "--noverbose"
]
view.main(view_opts)

title = "LOS Linear Velocity Fit"
view_opts = [
    vel_file, "velocity",
    "-m", msk_file,
    "--dem", f"{mintpy_dir}/inputs/geometryGeo.h5", "--dem-blend", "--dem-nocontour",
    "--lalo-label",
    "--coastline", "10m",
    "-c", cmap_str,
    "-u", "mm/yr",
    "-v", str(vmin), str(vmax),
    "--title", title,
    "--noverbose"
]
view.main(view_opts)

<a id='find_gps'></a>
### 4.3. Find Collocated GNSS Stations

The project will have access to L2 position data for continuous GNSS stations in third-party networks such NSF’s Plate Boundary Observatory, the HVO network for Hawaii, GEONET-Japan, and GEONET-New Zealand, located in target regions for NISAR solid earth calval. Station data will be post-processed by one or more analysis centers, will be freely available, and will have latencies of several days to weeks, as is the case with positions currently produced by the NSF’s GAGE Facility and separately by the University of Nevada Reno. Networks will contain one or more areas of high-density station coverage (2~20 km nominal station spacing over 100 x 100 km or more) to support validation of L2 NISAR requirements at a wide range of length scales.

MintPy supports several options for downloading and handling GNSS data. Currently supported data sets are:
* UNR National Geodetic Laboratory
* SIO/JPL MEaSUREs ESESES
* JPL SIDESHOW

In [ ]:
# GNSS processing source
if 'gnss_source' in sitedata['sites'][site]:
    gnss_source = sitedata['sites'][site]['gnss_source']
else:
    gnss_source = 'UNR'
print(f"GNSS processing source: {gnss_source:s}")

# Get analysis metadata from InSAR velocity file
insar_metadata = readfile.read_attribute(vel_file)
lat_step = float(insar_metadata['Y_STEP'])
lon_step = float(insar_metadata['X_STEP'])
(S,N,W,E) = ut.four_corners(insar_metadata)
start_date = insar_metadata.get('START_DATE', None).split('T')[0]
end_date = insar_metadata.get('END_DATE', None).split('T')[0]
start_date_gnss = dt.strptime(start_date, "%Y%m%d")
end_date_gnss = dt.strptime(end_date, "%Y%m%d")

# Get center lat/lon of analysis region
bbox = sitedata['sites'][site]['analysis_region'].replace("'","").split()
lat0 = (float(bbox[0]) + float(bbox[1]))/2
lon0 = (float(bbox[2]) + float(bbox[3]))/2

# Identify geometry file x/y location associated with center lat/lon
geom_obj = mintpy_dir + '/inputs/geometryGeo.h5'
atr = readfile.read_attribute(geom_obj)
coord = ut.coordinate(atr, lookup_file=geom_obj)
y, x = coord.geo2radar(lat0, lon0)[0:2]
y = max(0, y);  y = min(int(atr['LENGTH'])-1, y)
x = max(0, x);  x = min(int(atr['WIDTH'])-1, x)

# Write out inclination/azimuth at specified x/y location
kwargs = dict(box=(x,y,x+1,y+1))
inc_angle = readfile.read(geom_obj, datasetName='incidenceAngle', **kwargs)[0][0,0]
az_angle  = readfile.read(geom_obj, datasetName='azimuthAngle',   **kwargs)[0][0,0]

# Set GNSS Parameters
gnss_completeness_threshold = 0.9    #0.9  #percent of data timespan with valid GNSS epochs
# max threshold standard deviation of residuals to linear GNSS fit not specified

# Search for collocated GNSS stations
site_names, _, _ = gnss.search_gnss(SNWE=(S,N,W,E),
                                    start_date=start_date,
                                    end_date=end_date,
                                    source=gnss_source)
site_names = [str(stn) for stn in site_names]

<a id='gps_ts'></a>
### 4.4. Get GNSS Position Time Series

We download the GNSS station data, and evaluate the station data quality based on data completeness and scatter of residuals.

In [ ]:
# Empty dicts and lists to store GNSS data
gnss_stns = {}
bad_stns = {}

print(f"GNSS completion threshold: {gnss_completeness_threshold}")

for site_name in site_names:
    gnss_stn = gnss.get_gnss_class(gnss_source)(site = site_name)
    gnss_stn.open(print_msg=False)

    # check if station is in masked out area
    site_lat, site_lon = gnss_stn.get_site_lat_lon()
    site_y, site_x = ut.coordinate(insar_metadata).geo2radar(site_lat, site_lon)[:2]
    masked_loc = np.isnan(insar_velocities[site_y, site_x])

    # count number of dates in time range
    dates = gnss_stn.dates
    range_days = (end_date_gnss - start_date_gnss).days
    gnss_count = np.histogram(dates, bins=[start_date_gnss, end_date_gnss])
    gnss_count = int(gnss_count[0][0])

    # get standard deviation of residuals to linear fit
    G = time_func.get_design_matrix4time_func(gnss_stn.date_list, ts_functions)
    disp_los = ut.enu2los(gnss_stn.dis_e, gnss_stn.dis_n, gnss_stn.dis_u, inc_angle, az_angle)
    m_hat = np.linalg.pinv(G).dot(disp_los)
    dis_hat = np.dot(G, m_hat)
    stn_stdv = np.std(disp_los - dis_hat)
    
    # select GNSS stations based on data completeness, scatter of residuals
    # and whether they are in masked areas
    if (range_days*gnss_completeness_threshold <= gnss_count) \
        and (masked_loc == False):
        gnss_stns[site_name] = gnss_stn
    else:
        bad_stns[site_name] = gnss_stn

# [optional] manually remove additional stations
gnss_to_remove=[]

for site_name in gnss_to_remove:
    bad_stns[site_name] = gnss_stns[site_name]
    del gnss_stns[site_name]

# Final list of site names
site_names = [*gnss_stns]
bad_site_names = [*bad_stns]

print("\nFinal list of {} stations used in analysis:".format(len(site_names)))
print(site_names)
print("List of {} stations removed from analysis".format(len(bad_site_names)))
print(bad_site_names)

In [ ]:
# Plot stations in map view
fig, ax = plt.subplots(figsize=[18, 5.5])
cax = ax.imshow(EQstep, cmap=cmap, vmin=vmin, vmax=vmax, interpolation='nearest', extent=(W, E, S, N))
cbar = fig.colorbar(cax, ax=ax)
cbar.set_label('LOS velocity [mm/year]')

# Plot valid stations
for i, gnss_stn in enumerate(gnss_stns.values()):
    # Plot station
    ax.scatter(gnss_stn.site_lon, gnss_stn.site_lat, s=8**2, c='k',
               label=('valid' if i == 0 else None))
    ax.annotate(gnss_stn.site, (gnss_stn.site_lon, gnss_stn.site_lat))

# Plot rejected stations
for i, gnss_stn in enumerate(bad_stns.values()):
    # Plot station
    ax.scatter(gnss_stn.site_lon, gnss_stn.site_lat, s=8**2,
               marker='X', facecolor='none', edgecolor='k',
              label=('rejected' if i == 0 else None))
    ax.annotate(gnss_stn.site, (gnss_stn.site_lon, gnss_stn.site_lat))

ax.legend()

<a id='gps_los'></a>
### 4.5. Make GNSS LOS Velocities/Displacements

One of our goals is to compare the coseismic displacements computed from InSAR, to those observed using continuous GNSS measurements. To make a direct comparison, we first projec the 3D GNSS displacement series into InSAR LOS (using the local incidence and azimuth angles). We then estimate the coseismic steps recorded by the GNSS using two methods:

1. We calculate a simple difference in GNSS station position at two dates bracketing the earthquake. This will likely be a noisy estimate.
2. We fit the LOS-projected GNSS displacement time-series using the same basis functions (i.e., polynomial and step term) that were used to fit the InSAR time-series. For this latter process, outliers are removed from the GNSS position data iteratively and the functional fits are recomputed. This should ensure the most robust estimate of coseismic displacements.

In [ ]:
ts_functions

In [ ]:
# Step date index
step_ndx = ts_functions['polynomial'] + len(ts_functions['periodic']) + 1
print(f"Step function parameter at index {step_ndx:d}")

# Outlier ID and removal parameters
outlier_thresh = 3  # standard deviations
outlier_iters = 1  # iterations

# Loop through valid GNSS stations
gnss_fits = {}  # empty dict to store fits
gnss_sites = {}  # empty dict to store site measurements

for i, site_name in enumerate(site_names):
    # Retrieve station information
    gnss_stn = gnss_stns[site_name]
    gnss_stn.get_los_displacement(insar_metadata,
                                  start_date=start_date,
                                  end_date=end_date)

    # Scale displacement values of m to mm
    scale_gnss_m_to_mm(gnss_stn)

    # Outlier detection and removal
    gnss_fit = IterativeOutlierFit(gnss_stn.dates, gnss_stn.dis_los,
                                   model=ts_functions, threshold=outlier_thresh,
                                   max_iter=outlier_iters)
    gnss_fits[site_name] = gnss_fit  # record for posterity

    # Record GNSS site velocity
    gnss_site = SiteDisplacement(site=site_name,
                                 site_lon=gnss_stn.site_lon,
                                 site_lat=gnss_stn.site_lat,
                                 dis=gnss_fit.m_hat[step_ndx],
                                 dis_err=gnss_fit.mhat_se[step_ndx],
                                 unit='mm')
    gnss_sites[site_name] = gnss_site

    # Report
    if i == 0 :
        print('site displacement(mm)')
    print(str(gnss_site))

    # Plotting parameters
    n_dates = len(gnss_stn.dates)
    label_skips = n_dates//6
    ax_nb = i % 2

    # Spawn figure for even numbers
    if ax_nb == 0:
        fig, axes = plt.subplots(figsize=(10, 4), ncols=2)

    # Plot outliers
    if gnss_fit.n_outliers > 0:
        axes[ax_nb].scatter(gnss_fit.outlier_dates,
                            gnss_fit.outlier_dis,
                            2**2, 'firebrick', label='outliers')

    # Plot filtered data and model fit
    axes[ax_nb].scatter(gnss_fit.dates, gnss_fit.dis, 3**2,
                        'dimgrey', zorder=1)
    axes[ax_nb].plot(gnss_fit.dates, gnss_fit.dis_hat,
                     'c', linewidth=3, label='model fit', zorder=2)
    axes[ax_nb].plot(gnss_fit.dates, gnss_fit.err_envelope[0],
                     'b--', zorder=3)
    axes[ax_nb].plot(gnss_fit.dates, gnss_fit.err_envelope[1],
                     'b--', zorder=3, label='95% conf')

    # Format plot
    axes[ax_nb].legend()
    axes[ax_nb].set_xticks(gnss_stn.dates[::label_skips])
    axes[ax_nb].set_xticklabels([date.strftime('%Y-%m-%d') \
                                 for date in gnss_stn.dates[::label_skips]],
                                rotation=80)
    axes[ax_nb].set_ylabel('LOS displacement (mm)')
    axes[ax_nb].set_title(str(gnss_site))
    fig.tight_layout()

<a id='reference'></a>
### 4.6. Re-reference GNSS and InSAR LOS Coseismic Step


In [ ]:
# Reference GNSS stations to reference site
ref_site = sitedata['sites'][site]['gps_ref_site_name']
if sitedata['sites'][site]['gps_ref_site_name'] == 'auto':
    ref_site = [*gnss_sites.keys()][0]

gnss_ref_dis = gnss_sites[ref_site].dis
print(f"Using GNSS reference station: {ref_site}")

for site_name in site_names:
    gnss_sites[site_name].dis -= gnss_ref_dis
    print(str(gnss_sites[site_name]))

# Reference InSAR to GNSS reference site
(ref_y,
 ref_x,
 _, _) = ut.coordinate(insar_metadata).geo2radar(gnss_sites[ref_site].site_lat,
                                                 gnss_sites[ref_site].site_lon)

insar_ref_step = EQstep[ref_y, ref_x]
EQstep = EQstep - insar_ref_step
print(f"Insar reference step {insar_ref_step:.1f} mm")

# Plot GNSS stations on InSAR velocity field
gnss_insar_fig, ax = plt.subplots(figsize=[18, 5.5])
cax = ax.imshow(EQstep, cmap=cmap, vmin=vmin, vmax=vmax,
                interpolation='nearest', extent=(W, E, S, N))
cbar = gnss_insar_fig.colorbar(cax, ax=ax)
cbar.set_label('LOS EQ displacement [mm]')

for site_name in site_names:
    gnss_site = gnss_sites[site_name]
    color = cmap((gnss_site.dis - vmin)/(vmax-vmin))
    ax.scatter(gnss_site.site_lon, gnss_site.site_lat, color=color, s=8**2, edgecolors='k')
    ax.text(gnss_site.site_lon, gnss_site.site_lat, gnss_site.site)
    ax.set_title(f"{gnss_source} GNSS step on InSAR step")

<br>
<hr>

<a id='validation1'></a>
## 5. NISAR Validation Approach 1: GNSS-InSAR Direct Comparison 


<a id='make_resid'></a>
### 5.1. Make GNSS-InSAR Displacement Residuals at GNSS Station Locations


To test how well the InSAR measurements agree with the GNSS, we extract the InSAR displacement value at each GNSS site location and compute the differences (residuals) between the two data sets .

In [ ]:
# Set Parameters
pixel_radius = 5   #number of InSAR pixels to average for comparison with GNSS

# Loop over InSAR measurements at GNSS station locations
insar_sites = {}  # empty dict to store site measurements

for i, site_name in enumerate(site_names):
    gnss_site = gnss_sites[site_name]
    
    # Convert GNSS station lat/lon information to InSAR x/y grid
    stn_y, stn_x, _, _ = coord.geo2radar(gnss_site.site_lat,
                                         gnss_site.site_lon)

    # Caution: If you expand the radius parameter farther than the bounding grid it will break. 
    # To fix, remove the station in section 4 when the site_names list is filtered
    dis_px_rad = EQstep[stn_y-pixel_radius:stn_y+1+pixel_radius,
                        stn_x-pixel_radius:stn_x+1+pixel_radius]
    insar_site_dis = np.nanmedian(dis_px_rad)

    # Assign to object
    insar_site = SiteDisplacement(site=site_name,
                              site_lon=gnss_site.site_lon,
                              site_lat=gnss_site.site_lat,
                              dis=insar_site_dis,
                              unit='mm')
    insar_sites[site_name] = insar_site

    # Report
    if i == 0 :
        print('site displacement(mm)')
    print(str(insar_site))

<a id='make_ddiff'></a>
### 5.2. Make Double-differenced Displacement Residuals (from Step Function Approach)


We then compute the "double difference" residuals between pairs of collocated InSAR and GNSS measurements. The double difference effectively cancels out any remaining referencing errors between the InSAR and GNSS data.

In [ ]:
# Compute double differences between site pairs
n_sites = len(site_names)

# Loop over stations
double_diffs = {}  # empty dict to populate with InSAR - GNSS diffs
for i in range(n_sites - 1):
    stn1 = site_names[i]
    for j in range(i + 1, n_sites):
        stn2 = site_names[j]

        # Compute distance (km) between site locations
        site_dist = haversine_distance(gnss_sites[stn1].site_lon,
                                       gnss_sites[stn1].site_lat,
                                       gnss_sites[stn2].site_lon,
                                       gnss_sites[stn2].site_lat)
        
        # Calculate site-to-site differences
        diff_name = f"{stn1}-{stn2}"
        gnss_diff = gnss_sites[stn1] - gnss_sites[stn2]
        insar_diff = insar_sites[stn1] - insar_sites[stn2]
        diff_res = gnss_diff - insar_diff
        diff_res.dist = site_dist
        diff_res.site = diff_name

        # Record to dictionary
        double_diffs[diff_name] = diff_res

<a id='amp_vs_dist'></a>
### 5.3. Amplitude vs. Distance of Double-differences (not quite a structure function)

The coseismic requirement is that, for earthquakes that generate surface displacements &ge; 100 mm, at least 68.3% of surface displacements should be measured with an accuracy of $4(1 + \sqrt{L})$ or better, over length scales of $0.1 \mathrm{km} < L < 50 \mathrm{km}$, where $L$ is the distance from one sample point to another. We assume that the distribution of residuals is Gaussian and that the requirement success threshold represents a 1-sigma limit within which we expect 68.3% of residuals to lie.

In [ ]:
# Set requirement thresholds
coseismic_distance_rqmt = (0.1, 50)  # distances for evaluation
coseismic_threshold_rqmt = lambda L: 4 * (1 + np.sqrt(L))  # coseismic threshold in mm

n_bins = 10  # number of distance bins for analysis
threshold = 0.683  # fraction of Gaussian normal distribution for pass/fail

# Define validation parameters
sensor = 'Sentinel-1'
validation_data = 'GNSS'

# Write data for statistical tests
site_dist = np.array([diff_res.dist for diff_res in double_diffs.values()])
double_diff_rel_measure = np.abs(np.array([diff_res.dis for diff_res in double_diffs.values()]))
double_diff_rel_measure_err = np.array([diff_res.dis_err for diff_res in double_diffs.values()])

# Validation figure and assessment
site_loc = sitedata['sites'][site]['calval_location']
(validation_table_method1,
 validation_fig_method1) = display_coseismic_validation(site_dist,                           # binned distance for point
                                     double_diff_rel_measure,             # binned double-difference velocities mm/yr
                                     site_loc,                                # cal/val site name
                                     start_date,                          # start date of InSAR dataset
                                     end_date,                            # end date of InSAR dataset 
                                     requirement=coseismic_threshold_rqmt,  # measurement requirement to meet, e.g 2 mm/yr for 3 years of data over 0.1-50km
                                     distance_rqmt=coseismic_distance_rqmt, # distance over requirement is to meet, e.g. over length scales of 0.1-50 km [0.1, 50] 
                                     n_bins=n_bins,                       # number of bins, to collect statistics 
                                     threshold=threshold,                 # quantile threshold for point-pairs that pass requirement, e.g. 0.683 - we expect 68.3% of residuals to lie. 
                                     sensor='Sentinel-1',                 # sensor that is validated, Sentinel-1 or NISAR
                                     validation_type=requirement.lower(), # validation for: secular, transient, coseismic requirement
                                     validation_data='GNSS (step)') # validation method: GNSS - Method 1, InSAR - Method 2

In [ ]:
display_validation_table(validation_table_method1)

<div class="alert alert-warning">
Final result Method 1—Succesful when 68.3% of points below requirements line
</div>


In [ ]:
# Save Method 1 results to file
run_date = dt.now().strftime('%Y%m%dT%H%M%S')
save_fldr = f"{run_date}-Coseismic-Method1"
save_dir = os.path.join(mintpy_dir, save_fldr)

save_params = {
    'save_dir': save_dir,
    'run_date': run_date,
    'requirement': "Coseismic",
    'site': site,
    'method': "1",
    'sitedata': sitedata['sites'][site],
    'gnss_insar_figs': [gnss_insar_fig],
    'validation_figs': [validation_fig_method1],
    'validation_table': validation_table_method1,
    'ts_functions': ts_functions,
}
save_results(**save_params)

# Save the report in the home directory as well
save_params['save_dir'] = home_dir
save_results(**save_params)

<a id='validation2'></a>
## 6. NISAR Validation Approach 2: InSAR-only Structure Function

In Validation Approach 2, we use a date when there was no earthquake and do the same step function fit.

In [ ]:
# Check that an earthquake date is specified and retrieve that date
if 'noEarthquakeDate' in sitedata['sites'][site].keys():
    noEQdate = [sitedata['sites'][site]['noEarthquakeDate']]
    print('No-earthquake date: {:s}'.format(sitedata['sites'][site]['noEarthquakeDate']))
else:
    noEQdate = []
    warnings.warn('No non-earthquake date specified. This data set might not be suitable for the coseismic workflow.')

In [ ]:
os.chdir(mintpy_dir)  # reset directory in case running out of sequence
print(f"Present directory: {mintpy_dir:s}")

# Step function fit when there was no earthquake
vel_file = "velocity_noearthquake.h5"
command = 'timeseries2velocity.py ' + timeseries_filename + ' -o ' + vel_file + ' --step ' + sitedata['sites'][site]['noEarthquakeDate']
process = subprocess.run(command, shell=True)

Now we can view the step function amplitude for the non-earthquake date and the new linear velocity estimate. Because we have the step function at a time far away from the earthquake, the step function has a small amplitude and the linear velocity has absorbed the time-series displacements. The non-earthquake step plot has the same color scale as the earthquake coseismic plot above to show the small atmospheric noise of a date without an earthquake.

In [ ]:
# Visualize velocity
title = "LOS Coseismic, no earthquake date"
view_opts = [
    vel_file, "step"+sitedata['sites'][site]['noEarthquakeDate'],
    "-m", msk_file,
    "--dem", f"{mintpy_dir}/inputs/geometryGeo.h5", "--dem-blend", "--dem-nocontour",
    "--lalo-label",
    "--coastline", "10m",
    "-c", cmap_str,
    "-u", "mm",
    "-v", str(vmin), str(vmax),
    "--title", title,
    "--noverbose"
]
view.main(view_opts)

title = "LOS Linear Velocity Fit"
view_opts = [
    vel_file, "velocity",
    "-m", msk_file,
    "--dem", f"{mintpy_dir}/inputs/geometryGeo.h5", "--dem-blend", "--dem-nocontour",
    "--lalo-label",
    "--coastline", "10m",
    "-c", cmap_str,
    "-u", "mm/yr",
    "-v", str(vmin), str(vmax),
    "--title", title,
    "--noverbose"
]
view.main(view_opts)

<a id='array_mask'></a>
### 6.1. Read Array and Mask Pixels with no Data

In [ ]:
# Use the non-earthquake displacement as the insar_displacment for statistics and convert to mm
noEQdataset = 'step' + sitedata['sites'][site]['noEarthquakeDate']
noEQstep, insar_metadata = readfile.read(vel_file, datasetName = noEQdataset)  #read coseismic step 
(S,N,W,E) = ut.four_corners(insar_metadata)
noEQdate = sitedata['sites'][site]['noEarthquakeDate']

insar_displacement = noEQstep * 1000. # convert from m to mm and add dimension to array to allow multiple datasets

ifgs_date = np.array([noEQdate])  # only one non-earthquake date for now
n_ifgs = insar_displacement.shape[0]

# Mask out no-data areas
msk, _ = readfile.read(msk_file)
insar_displacement[msk == 0] = np.nan
insar_displacement[insar_displacement == 0] = np.nan

# Display map of data after masking
fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.imshow(insar_displacement, vmin=vmin, vmax=vmax, cmap=cmap, interpolation='nearest', extent=(W, E, S, N))
ax.set_title("Coseismic \n Date "+noEQdate)
cbar1 = fig.colorbar(img1, ax=ax)
cbar1.set_label('LOS displacement [mm]')

<a id='remove_trend'></a>
### 6.2. Randomly Sample Pixels and Pair Them Up with Option to Remove Trend

First, calculate the coordinate for every pixel.

Then for each non-earthquake step fit, randomly selected pixels need to be paired up. In order to keep measurements independent, different pixel pairs can not share same pixel. This is achieved by pairing up in sequence, i.e., pairing up pixel number 1 and number 2, 3 and 4...

To pass the requirement, the pixel-to-pixels differences must satisfy the same $4(1 + \sqrt{L})$ or better, over length scales of $0.1 \mathrm{km} < L < 50 \mathrm{km}$ requirement specified above.

In [ ]:
# Set requirement thresholds
coseismic_distance_rqmt = (0.1, 50)  # distances for evaluation
coseismic_threshold_rqmt = lambda L: 4 * (1 + np.sqrt(L))  # coseismic threshold in mm

n_bins = 10  # number of distance bins for analysis
threshold = 0.683  # fraction of Gaussian normal distribution for pass/fail

sensor = 'Sentinel-1'
validation_data = 'InSAR'

In [ ]:
sample_mode = 'points'  # 'points' or 'profile'
# note that the 'profile' method may take significantly longer

# Collect samples using the specified method
if sample_mode in ['points']:
    X0,Y0 = load_geo(insar_metadata)
    X0_2d,Y0_2d = np.meshgrid(X0,Y0)

    insar_sample_dist, insar_rel_measure = samp_pair(X0_2d, Y0_2d, insar_displacement, num_samples=1000000)

elif sample_mode in ['profile']:
    # Sample grid setup
    length, width = int(insar_metadata['LENGTH']), int(insar_metadata['WIDTH'])
    X = np.linspace(W+lon_step, E-lon_step, width)  # longitudes
    Y = np.linspace(N+lat_step, S-lat_step, length)  # latitudes
    X_coords, Y_coords = np.meshgrid(X, Y)

    # Draw random samples from map (without replacement)
    num_samples = 20000
    
    # Retrieve profile samples
    insar_sample_dist, insar_rel_measure = profile_samples(\
                    x=X_coords.reshape(-1,1),
                    y=Y_coords.reshape(-1,1),
                    data=insar_displacement,
                    metadata=insar_metadata,
                    len_rqmt=secular_distance_rqmt,
                    num_samples=num_samples)

Check the statistical property of selected pixel pairs and overall histogram of relative measurements

In [ ]:
fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.hist(insar_sample_dist, bins=100)
ax.set_title(f"Histogram of distance \n Coseismic Date "+noEQdate)
ax.set_xlabel(r'Distance ($km$)')
ax.set_ylabel('Frequency')
ax.set_xlim(0,50)
    
fig, ax = plt.subplots(figsize=[18, 5.5])
img1 = ax.hist(insar_rel_measure, bins=100)
ax.set_title(f"Histogram of Relative Measurement \n Coseismic Date "+noEQdate)
ax.set_xlabel(r'Relative Measurement ($mm$)')
ax.set_ylabel('Frequency')

<a id='validation2'></a>
### 6.3. Coseismic Requirement Validation: Method 2


In approach 2, the number of pixel pairs which meet the mission requirement as a percentage of the total number of pixel pairs selected are counted.

The method we apply to evaluate the noise structure is similar to that in the InSAR-GNSS comparison. We count the percentage of measurements that fall below the threshold curve  for each of the 5-km-wide bins. If the average of the percentages from all bins is larger than 0.683, we judge that the noise level falls below the requirement.

Then we prepare table of results.

In [ ]:
# Validation figure and assessment
site_loc = sitedata['sites'][site]['calval_location']
(validation_table_method2,
 validation_fig_method2) = display_coseismic_validation(insar_sample_dist,                   # binned distance for point
                                     insar_rel_measure,                   # binned double-difference velocities mm/yr
                                     site_loc,                                # cal/val site name
                                     start_date,                          # start date of InSAR dataset
                                     end_date,                            # end date of InSAR dataset 
                                     requirement=coseismic_threshold_rqmt,  # measurement requirement to meet, e.g 2 mm/yr for 3 years of data over 0.1-50km
                                     distance_rqmt=coseismic_distance_rqmt, # distance over requirement is to meet, e.g. over length scales of 0.1-50 km [0.1, 50] 
                                     n_bins=n_bins,                       # number of bins, to collect statistics 
                                     threshold=threshold,                 # quantile threshold for point-pairs that pass requirement, e.g. 0.683 - we expect 68.3% of residuals to lie. 
                                     sensor='Sentinel-1',                 # sensor that is validated, Sentinel-1 or NISAR
                                     validation_type=requirement.lower(), # validation for: secular, transient, coseismic requirement
                                     validation_data=validation_data)     # validation method: GNSS - Method 1, InSAR - Method 2

out_fig = os.path.abspath('coseismic_insar-only_vs_distance_'+site+'_date'+noEQdate+'.png')
fig.savefig(out_fig, bbox_inches='tight', transparent=True, dpi=300)

In [ ]:
display_validation_table(validation_table_method2)

<div class="alert alert-warning">
Final result Method 2 table of distance bins—
    68% of points below the requirements line is success
</div>


In [ ]:
# Save Method 2 results to file
run_date = dt.now().strftime('%Y%m%dT%H%M%S')
save_fldr = f"{run_date}-Coseismic-Method2"
save_dir = os.path.join(mintpy_dir, save_fldr)

save_params = {
    'save_dir': save_dir,
    'run_date': run_date,
    'requirement': "Coseismic",
    'site': site,
    'method': "2",
    'sitedata': sitedata['sites'][site],
    'gnss_insar_figs': [gnss_insar_fig],
    'validation_figs': [validation_fig_method2],
    'validation_table': validation_table_method2,
    'ts_functions': ts_functions,
}
save_results(**save_params)

# Save the report in the home directory as well
save_params['save_dir'] = home_dir
save_results(**save_params)

<a id='appendix'></a>
## Appendix: Supplementary Comparisons and Plots


<a id='compare_raw'></a>
### A.1. Compare Raw Displacements


In [ ]:
plt.figure(figsize=(10,3))
plt.hist([insar_site.dis for insar_site in insar_sites.values()],
         range=[vmin/2, vmax/2], bins=50, color="green", edgecolor='grey', label='V_InSAR')
plt.hist([gnss_site.dis for gnss_site in gnss_sites.values()],
         range=[vmin/2, vmax/2], bins=50, color="orange", edgecolor='grey', label='V_gnss', alpha=0.5)
plt.legend(loc='upper right')
plt.title(f'''Displacements \n Date range {start_date}-{end_date}
Reference stn: {sitedata['sites'][site]['gps_ref_site_name']}
Number of stations used: {len(site_names)}''')
plt.xlabel('LOS Displacement (mm)')
plt.ylabel('N Stations')
plt.show()

<a id='plot_residuals'></a>
### A.2. Plot Displacement Residuals


In [ ]:
plt.figure(figsize=(10,3))
plt.hist([(insar_sites[site_name] - gnss_sites[site_name]).dis for site_name in site_names],
         bins = 40, range=[vmin/2, vmax/2], edgecolor='grey', color="darkblue", linewidth=1,
         label='V_gnss - V_InSAR (area average)')
plt.legend(loc='upper right')
plt.title(f'''Residuals \n Date range {start_date}-{end_date}
Reference stn: {sitedata['sites'][site]['gps_ref_site_name']}
Number of stations used: {len(site_names)}''')
plt.xlabel('Displacement Residual (mm)')
plt.ylabel('N Stations')
plt.show()

<a id='plot_ddiff'></a>
### A.3. Plot Double-differenced Residuals


In [ ]:
plt.figure(figsize=(10,3))
plt.hist([diff_res.dis for diff_res in double_diffs.values()],
         range = [vmin/2, vmax/2],bins = 40, color = "darkblue",edgecolor='grey',
         label='V_gnss_(s1-s2) - V_InSAR_(s1-s2)')
plt.legend(loc='upper right')
plt.title(f'''Difference Residualts \n Date range {start_date}-{end_date}
Reference stn: {sitedata['sites'][site]['gps_ref_site_name']}
Number of stations used: {len(site_names)}''')
plt.xlabel('Double Differenced Velocity Residual (mm)')
plt.ylabel('N Stations')
plt.show()

<a id='plot_timeseries'></a>
### A.4. GNSS Time-series Plots


In [ ]:
# Grab the time-series file used for time function estimation given the template setup
template = readfile.read_template(os.path.join(mintpy_dir, 'smallbaselineApp.cfg'))
template = ut.check_template_auto_value(template)
ts_file = TimeSeriesAnalysis.get_timeseries_filename(template, mintpy_dir)['velocity']['input']

# Read the time-series file
insar_ts, ts_metadata = readfile.read(ts_file, datasetName='timeseries')
insar_ts *= 1000  # meter to mm
mask = readfile.read(os.path.join(mintpy_dir, 'maskTempCoh.h5'))[0]
print(f"reading timeseries from file: {ts_file}")

# Get date list
date_list = timeseries(ts_file).get_date_list()
num_date = len(date_list)
date0, date1 = date_list[0], date_list[-1]
insar_dates = ptime.date_list2vector(date_list)[0]

# Spatial reference
coord = ut.coordinate(ts_metadata)
ref_site = sitedata['sites'][site]['gps_ref_site_name']
if sitedata['sites'][site]['gps_ref_site_name'] == 'auto':
    ref_site = [*gnss_sites.keys()][0]
ref_gnss_obj = gnss_stns[ref_site]
ref_lat, ref_lon = ref_gnss_obj.get_site_lat_lon()
ref_y, ref_x = coord.geo2radar(ref_lat, ref_lon)[:2]
if not mask[ref_y, ref_x]:
    raise ValueError(f'Given reference GNSS site ({ref_site}) is in mask-out unrelible region in InSAR! Change to a different site.')
ref_insar_dis = insar_ts[:, ref_y, ref_x]

# Plot displacements and velocity timeseries at GNSS station locations
num_site = len(site_names)
prog_bar = ptime.progressBar(maxValue=num_site)
for i, site_name in enumerate(site_names):
    prog_bar.update(i+1, suffix=f'{site_name} {i+1}/{num_site}')

    ## Read data
    # Recall gnss station displacements with outliers removed
    gnss_obj = gnss_stns[site_name]
    gnss_lalo = (gnss_obj.site_lat, gnss_obj.site_lon)

    # Get relative LOS displacement on common dates
    gnss_dates = np.array(sorted(list(set(gnss_obj.dates) & set(ref_gnss_obj.dates))))
    gnss_dis = np.zeros(gnss_dates.shape, dtype=np.float32)
    for i, date_i in enumerate(gnss_dates):
        idx1 = np.where(gnss_obj.dates == date_i)[0][0]
        idx2 = np.where(ref_gnss_obj.dates == date_i)[0][0]
        gnss_dis[i] = gnss_obj.dis_los[idx1] - ref_gnss_obj.dis_los[idx2]
    
    # Shift GNSS to zero-mean in time [for plotting purpose]
    gnss_dis -= np.nanmedian(gnss_dis)

    # Read InSAR
    y, x = coord.geo2radar(gnss_lalo[0], gnss_lalo[1])[:2]
    insar_dis = insar_ts[:, y, x] - ref_insar_dis

    # Apply a constant shift in time to fit InSAR to GNSS
    comm_dates = sorted(list(set(gnss_dates) & set(insar_dates)))
    if comm_dates:
        insar_flag = [x in comm_dates for x in insar_dates]
        gnss_flag = [x in comm_dates for x in gnss_dates]
        insar_dis -= np.nanmedian(insar_dis[insar_flag] - gnss_dis[gnss_flag])

    ## Plot figure
    if gnss_dis.size > 0 and np.any(~np.isnan(insar_dis)):
        fig, ax = plt.subplots(figsize=(10, 3))
        ax.axhline(color='grey',linestyle='dashed', linewidth=2)
        ax.scatter(gnss_dates, gnss_dis, s=2**2, label="GNSS Daily Positions")
        ax.scatter(insar_dates, insar_dis, label="InSAR Positions")
        # axis format
        ax.set_title(f"Station Name: {site_name}") 
        ax.set_ylabel('LOS displacement [mm]')
        ax.legend()
prog_bar.close()
plt.show()